In [26]:
import cv2
import numpy as np

# read image
img = cv2.imread('../doho.png')
tmplt_handle = cv2.imread('template_open2.png', cv2.IMREAD_UNCHANGED)
tmplt_deadbolt = cv2.imread('template_deadbolt.png', cv2.IMREAD_UNCHANGED)


def getMatch(img, tmplt):

# read template with alpha
    hh, ww = tmplt.shape[:2]
    tmplt_mask = tmplt[:,:,3]
    tmplt_mask = cv2.merge([tmplt_mask,tmplt_mask,tmplt_mask])
    tmplt2 = tmplt[:,:,0:3]
    corrimg = cv2.matchTemplate(img,tmplt2,cv2.TM_CCORR_NORMED, mask=tmplt_mask)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corrimg)
    max_val_ncc = '{:.3f}'.format(max_val)
    print("correlation match score: " + max_val_ncc)
    xx = max_loc[0]
    yy = max_loc[1]
    print('xmatch =',xx,'ymatch =',yy)

    # draw red bounding box to define match location
    result = img.copy()
    pt1 = (xx,yy)
    pt2 = (xx+ww, yy+hh)
    cv2.rectangle(result, pt1, pt2, (0,0,255), 1)
    newimg = img[pt1[1]:pt2[1], pt1[0]:pt2[0]]
    newimg = newimg[len(newimg)//3:2*len(newimg)//3, len(newimg[0])//3:2*len(newimg[0])//3]
    cv2.imshow('result', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return newimg

# save results
getMatch(img, tmplt_deadbolt)



correlation match score: 0.998
xmatch = 446 ymatch = 194


array([[[116, 132, 139],
        [124, 139, 146],
        [122, 138, 145],
        [122, 138, 147],
        [125, 142, 151],
        [128, 145, 153],
        [142, 160, 168],
        [144, 163, 170],
        [147, 165, 172],
        [150, 168, 175],
        [149, 167, 174],
        [150, 168, 175],
        [142, 159, 166],
        [137, 155, 162],
        [140, 157, 164],
        [148, 163, 172]],

       [[105, 121, 128],
        [113, 128, 135],
        [114, 130, 138],
        [114, 130, 139],
        [119, 136, 145],
        [127, 145, 153],
        [134, 152, 160],
        [143, 161, 169],
        [144, 162, 169],
        [145, 162, 169],
        [141, 159, 165],
        [148, 166, 173],
        [141, 159, 165],
        [136, 153, 160],
        [134, 150, 158],
        [130, 144, 154]],

       [[192, 205, 213],
        [189, 203, 210],
        [156, 170, 178],
        [123, 137, 145],
        [119, 134, 142],
        [118, 134, 142],
        [130, 147, 153],
        [130, 148, 15